# 🚀 고객 세분화를 위한 RFM 분석 (고객 성향 데이터 활용)

데이터 기반 마케팅에서 가장 중요한 질문 중 하나는 '누가 우리의 진짜 고객인가?' 입니다. 모든 고객을 동일하게 대하는 것은 비효율적이며, 고객의 특성에 맞는 맞춤형 전략이 필요합니다. **RFM 분석**은 이러한 고객 세분화(Customer Segmentation)를 위한 가장 고전적이면서도 강력한 데이터 분석 기법입니다.

이번 시간에는 고객의 인구통계 및 소비 패턴이 정리된 **'고객 성향 분석(Customer Personality Analysis)'** 데이터셋을 활용하여 RFM 지표를 생성하고, 이를 바탕으로 고객을 의미 있는 그룹으로 분류하는 전체 과정을 함께 진행해 보겠습니다.

### 1. RFM 분석이란?

#### 🧠 개념 이해하기

RFM은 고객의 가치를 평가하기 위한 세 가지 지표의 약자입니다.

* **Recency (R) - 최신성**: 고객이 얼마나 **최근에** 구매했는가?
    * 최근에 구매한 고객일수록 다시 구매할 확률이 높습니다. 고객의 현재 관심도와 활성 상태를 나타냅니다.
* **Frequency (F) - 구매 빈도**: 고객이 얼마나 **자주** 구매했는가?
    * 자주 구매하는 고객일수록 브랜드에 대한 충성도가 높습니다. 고객의 충성도를 나타내는 핵심 지표입니다.
* **Monetary (M) - 구매 금액**: 고객이 얼마나 **많은 돈을** 지출했는가?
    * 많은 금액을 지출한 고객은 비즈니스의 수익에 크게 기여하는 VIP 고객일 가능성이 높습니다.

### 2. 데이터 준비 및 전처리

#### 💻 코드로 알아보기

이번에 사용할 데이터셋은 각 고객의 정보와 소비 활동이 요약된 형태로, 바로 RFM 지표를 생성하기에 용이한 구조입니다. 먼저 데이터를 불러오고 기본적인 전처리를 수행합니다.

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


In [10]:

# 제공된 경로와 구분자로 데이터 로드
path = '../datasets/ml/crm/marketing_campaign.csv'
df = pd.read_csv(path, sep='\t')
df.head()


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


| | 컬럼 | 설명 |
| --- | --- | --- |
| 0 | Invoice | 송장 번호 |
| 1 | StockCode | 재고 코드 |
| 2 | Description | 상품 설명 |
| 3 | Quantity | 수량 |
| 4 | InvoiceDate | 송장 날짜 |
| 5 | Price | 가격 |
| 6 | Customer ID | 고객 ID |
| 7 | Country | 국가 |


In [11]:
# Income 컬럼의 결측치를 중앙값으로 대체
df['Income'].fillna(df['Income'].median(), inplace=True)
print("전처리 완료. 결측치 없음.")

전처리 완료. 결측치 없음.


### 3. R, F, M 지표 생성 및 분포 확인

#### 🧠 개념 이해하기

이 데이터셋은 일반적인 거래 로그 데이터와 다르게 일부 지표가 이미 계산되어 있습니다. 이 특성을 활용하여 R, F, M 지표를 생성합니다.

- **Recency (R)**: 데이터셋에 `Recency` 컬럼(마지막 구매 후 경과일)이 이미 존재하므로, 이 값을 그대로 사용합니다.
- **Frequency (F)**: `NumWebPurchases`, `NumCatalogPurchases`, `NumStorePurchases` 등 여러 채널을 통한 총 구매 횟수를 합산하여 고객의 총 구매 빈도를 생성합니다.
- **Monetary (M)**: `MntWines`, `MntFruits` 등 여러 카테고리의 총 구매액을 합산하여 고객의 총 구매 금액을 생성합니다.

In [ ]:
# RFM 지표를 담을 새로운 데이터프레임 생성
rfm_df = pd.DataFrame()
rfm_df['ID'] = df['ID']

# Recency 값은 기존 컬럼을 그대로 사용
rfm_df['Recency'] = df['Recency']

# Frequency 계산 (모든 구매 채널의 횟수 합산)
freq_cols = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
rfm_df['Frequency'] = df[freq_cols].sum(axis=1)

# Monetary 계산 (모든 제품 카테고리 구매액 합산)
monetary_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
rfm_df['Monetary'] = df[monetary_cols].sum(axis=1)

print("생성된 RFM 데이터프레임:")
rfm_df.head()

생성된 RFM 데이터프레임:


,ID,Recency,Frequency,Monetary
0,5524,58,22,1617
1,2174,38,4,27
2,4141,26,20,776
3,6182,26,6,53
4,5324,94,14,422


#### 💻 코드로 알아보기: R, F, M 분포 확인

스코어링을 진행하기 전에, 우리가 만든 R, F, M 지표가 어떤 분포를 가지고 있는지 확인하는 것은 매우 중요합니다. 이는 우리가 어떤 스코어링 방식을 사용할지 결정하는 데 중요한 단서가 됩니다.

In [ ]:
rfm_df[['Recency', 'Frequency', 'Monetary']].describe()

,Recency,Frequency,Monetary
count,2240.000000,2240.000000,2240.000000
mean,49.109375,12.537054,605.798214
std,28.962453,7.205741,602.249288
min,0.000000,0.000000,5.000000
25%,24.000000,6.000000,68.750000
50%,49.000000,12.000000,396.000000
75%,74.000000,18.000000,1045.500000
max,99.000000,32.000000,2525.000000


**분포 분석:**

- **Recency**: 25%, 50%, 75% 값이 각각 24, 49, 74로 비교적 고르게 분포되어 있습니다.
- **Frequency & Monetary**: 75% 값과 max 값의 차이가 매우 큽니다. (예: Monetary의 75%는 약 1045인데, max는 2525). 이는 소수의 '헤비 유저' 또는 'VIP 고객'이 전체 평균을 높이는, 오른쪽으로 꼬리가 긴(right-skewed) 분포임을 시사합니다.

### 4. RFM 스코어 부여 및 세그먼트 정의

#### 🧠 개념 이해하기: 분위수(Quantile) 기반 스코어링

R, F, M 지표들의 분포와 단위가 다르기 때문에, 이를 공정한 기준으로 비교하기 위해 '점수'로 변환합니다. 가장 일반적인 방법은 **분위수(Quantile)**를 사용하는 것입니다. 예를 들어 사분위수(Quartile)는 전체 고객을 4개의 동일한 수의 그룹으로 나눕니다.

이 방식은 데이터의 분포가 한쪽으로 치우쳐져 있더라도, 각 그룹에 속하는 **고객의 수를 동일하게** 맞춰주기 때문에 마케팅 자원을 각 세그먼트에 균등하게 배분하고 관리하는 데 매우 유용합니다.

**점수 부여 규칙:**
-   **Recency (R)**: 값이 **작을수록** (더 최근일수록) 좋은 것이므로, 가장 높은 점수(4점)를 부여합니다.
-   **Frequency (F), Monetary (M)**: 값이 **클수록** 좋은 것이므로, 가장 높은 점수(4점)를 부여합니다.

In [ ]:
# R, F, M 값을 기준으로 4분위수(Quartile)로 나누어 점수 부여
r_labels = range(4, 0, -1)
f_labels = range(1, 5)
m_labels = range(1, 5)

rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], q=4, labels=r_labels, duplicates='drop')
rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'], q=4, labels=f_labels, duplicates='drop')
rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], q=4, labels=m_labels, duplicates='drop')

rfm_df['RFM_Score'] = rfm_df['R_Score'].astype(int) + rfm_df['F_Score'].astype(int) + rfm_df['M_Score'].astype(int)
rfm_df.head()

,ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,5524,58,22,1617,2,4,4,10
1,2174,38,4,27,3,1,1,5
2,4141,26,20,776,3,4,3,10
3,6182,26,6,53,3,1,1,5
4,5324,94,14,422,1,3,3,7


#### 🧠 심화 학습: 스코어링 방식과 데이터 분포

`pd.qcut`은 그룹 내 데이터의 '개수'를 동일하게 나눕니다. Monetary처럼 분포가 치우친 경우, 각 점수 그룹이 가지는 '값의 범위'는 어떻게 될까요?

In [ ]:
print("Monetary Score별 실제 값의 범위:")
print(rfm_df.groupby('M_Score')['Monetary'].agg(['min', 'max', 'mean', 'count']))

Monetary Score별 실제 값의 범위:
          min   max         mean  count
M_Score                                
1           5    68    39.262500    560
2          69   396   184.573975    561
3         397  1045   709.819320    559
4        1047  2525  1490.475000    560


/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/ipykernel_89598/1918117439.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



위 결과를 보면, `count`는 약 560명으로 모두 동일하지만, `max - min`으로 계산되는 값의 범위는 점수가 높을수록 극단적으로 넓어지는 것을 볼 수 있습니다. 특히 4점을 받은 최상위 그룹은 소수의 엄청난 VIP 고객을 포함하고 있음을 알 수 있습니다.

이처럼 `pd.qcut`은 고객 관리의 편의성을 위해 그룹의 '크기'를 맞추는 데 초점을 둡니다. 만약 '금액' 자체를 기준으로 구간을 나누고 싶다면(e.g., 100만원 이상은 무조건 VIP), `pd.cut`을 사용하여 직접 경계값을 설정하는 방법도 있습니다.

이제 총점을 기준으로 고객 세그먼트를 정의합니다.

In [ ]:
# RFM 점수 기반으로 세그먼트 정의
def assign_segment(score):
    if score >= 11:
        return 'Champions' # 최우수 고객
    elif score >= 9:
        return 'Loyalists' # 충성 고객
    elif score >= 6:
        return 'Potential' # 잠재 고객
    elif score >= 4:
        return 'Needs Attention' # 관심 필요 고객
    else:
        return 'At Risk' # 이탈 위험 고객

rfm_df['Segment'] = rfm_df['RFM_Score'].apply(assign_segment)
rfm_df.head()

,ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,5524,58,22,1617,2,4,4,10,Loyalists
1,2174,38,4,27,3,1,1,5,Needs Attention
2,4141,26,20,776,3,4,3,10,Loyalists
3,6182,26,6,53,3,1,1,5,Needs Attention
4,5324,94,14,422,1,3,3,7,Potential


### 5. RFM 세그먼트 시각화 및 분석

#### 💻 코드로 알아보기

분석 결과를 시각화하여 각 세그먼트의 분포와 특징을 한눈에 파악합니다. 막대 차트와 트리맵은 RFM 분석 결과를 시각화하는 데 매우 효과적입니다.

In [ ]:
# 세그먼트별 고객 수 계산 및 시각화
segment_counts = rfm_df['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']

fig_bar = px.bar(segment_counts, x='Segment', y='Count', 
                 title='고객 세그먼트별 분포',
                 color='Segment',
                 text_auto=True)
fig_bar.update_xaxes(categoryorder='total descending')
fig_bar.show()

In [ ]:
# 트리맵으로 시각화
# 각 사각형의 크기는 고객의 총 구매액(Monetary), 색상은 최근성(Recency)을 나타냅니다.
fig_treemap = px.treemap(rfm_df, path=['Segment'], 
                         values='Monetary',
                         color='Recency',
                         hover_data=['Frequency', 'Monetary', 'Recency'],
                         color_continuous_scale='RdYlGn_r',
                         title='세그먼트별 총 구매액 및 최근성')
fig_treemap.show()

**분석 결과:**

- **분포**: 'Champions' 와 'Potential' 그룹이 고객 수 기준으로 가장 큰 비중을 차지하고 있습니다. 비즈니스의 허리가 되는 잠재 고객층이 두텁다는 긍정적인 신호로 해석할 수 있습니다.
- **가치 및 활성도**: 트리맵에서 'Champions' 세그먼트가 가장 큰 면적과 가장 짙은 녹색을 띱니다. 이는 이들이 전체 매출에 가장 크게 기여할 뿐만 아니라, 가장 최근까지도 활발하게 활동하는 핵심 고객 그룹임을 의미합니다.
- **전략**: 'Champions' 그룹에게는 신제품 우선 체험, VIP 전용 이벤트 등 특별한 혜택으로 충성도를 유지해야 합니다. 'Potential' 그룹에게는 멤버십 혜택 강화나 구매 빈도를 높일 수 있는 프로모션을 통해 'Loyalists' 또는 'Champions'로 전환을 유도하는 전략이 필요합니다. 'Needs Attention' 그룹에게는 이들의 관심을 다시 끌기 위한 파격적인 할인 쿠폰이나 설문조사를 통한 불만 파악 등의 노력이 필요합니다.